### Basic RAG Implementation with Vector Store
- Vector Store using FAISS

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader= WebBaseLoader("https://python.langchain.com/docs/versions/v0_3/")

loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
document = loader.load()
document

[Document(metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='\n\n\n\n\nLangChain v0.3 | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(document)
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='LangChain v0.3 | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: 

In [5]:
## OpenAI Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

## Store in the FAISS Vector Store
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(documents,embeddings)

In [6]:
## Initializing the Model
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

In [ ]:
## Just a Example to get the context based on the user query from the Vector Store
user_query = "Migrate using langchain-cli"
retrieved_docs = vector_store.similarity_search(user_query)
print(retrieved_docs[0].page_content)

Migrate using langchain-cli​
The langchain-cli can help update deprecated LangChain imports in your code automatically.
Please note that the langchain-cli only handles deprecated LangChain imports and cannot
help to upgrade your code from pydantic 1 to pydantic 2.
For help with the Pydantic 1 to 2 migration itself please refer to the Pydantic Migration Guidelines.
As of 0.0.31, the langchain-cli relies on gritql for applying code mods.
Installation​
pip install -U langchain-clilangchain-cli --version # <-- Make sure the version is at least 0.0.31
Usage​
Given that the migration script is not perfect, you should make sure you have a backup of your code first (e.g., using version control like git).
The langchain-cli will handle the langchain_core.pydantic_v1 deprecation introduced in LangChain 0.3 as well
as older deprecations (e.g.,from langchain.chat_models import ChatOpenAI which should be from langchain_openai import ChatOpenAI),


In [8]:
## Creating the Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

In [9]:
## Creating the Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)
print(document_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10faaa6f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fa94530>, root_client=<openai.OpenAI object at 0x10c580a70>, root_async_client=<openai.AsyncOpenAI object at 0x10faaa750>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser() kwargs={} config={'run_name': 'stuff_documents_chain'} config_factori

In [10]:
## Create Retriver Chain to add Vectore Store to the chain as an interface

retriever = vector_store.as_retriever()

from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1193eecf0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
     

In [13]:
## Run the RAG Pipeline

result=retrieval_chain.invoke({"input":"Migrate using langchain-cli"})
print("\n\nAnswer:")
print(result['answer'])



Answer:
What is the role of the `langchain-cli` in updating LangChain imports?

The `langchain-cli` helps update deprecated LangChain imports in your code automatically. It handles deprecated imports such as those in `langchain_core.pydantic_v1` and others introduced in earlier versions of LangChain. However, it does not assist with migrating from Pydantic 1 to Pydantic 2.
